# 3.1 Introduction to Ensemble Learning and Random Forests

## Introduction

In Modules 1 and 2, we explored logistic regression with regularization for predicting student departure. While logistic regression is powerful and interpretable, it makes certain assumptions about the data (e.g., linear relationships between features and log-odds). In this module, we introduce **Random Forests**—a fundamentally different approach that makes no such assumptions.

Random Forests belong to a class of algorithms called **ensemble methods**, which combine multiple models to achieve better performance than any single model could. This notebook explains the theory behind ensemble learning and how Random Forests work.

### Learning Objectives

By the end of this notebook, you will be able to:

1. Explain the concept of ensemble learning and why it works
2. Describe the bootstrap aggregating (bagging) technique
3. Understand how decision trees serve as base learners
4. Explain how Random Forests combine bagging with feature randomness
5. Compare Random Forests to logistic regression for classification tasks

## 1. The Wisdom of Crowds: Ensemble Learning

### 1.1 Why Ensemble Methods Work

The fundamental insight behind ensemble methods is that **a group of diverse learners can outperform any individual learner**. This is analogous to the "wisdom of crowds" phenomenon:

- A single person guessing the number of jellybeans in a jar will likely be off
- The average of 100 guesses is often remarkably close to the truth

In machine learning terms:
- A single decision tree might make systematic errors
- Hundreds of trees, each trained slightly differently, can average out those errors

**Key Insight**: For ensemble methods to work, the individual models must:
1. Have some predictive power (be better than random guessing)
2. Make different errors (be diverse)

In [ ]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Simulate the wisdom of crowds
np.random.seed(42)
true_value = 500  # True number of jellybeans

# Individual guesses with some accuracy
n_guessers = 100
individual_guesses = np.random.normal(true_value, 100, n_guessers)  # Some variance

# Calculate running average as we add more guessers
ensemble_averages = np.cumsum(individual_guesses) / np.arange(1, n_guessers + 1)

fig = go.Figure()

# Individual guesses
fig.add_trace(go.Scatter(
    x=np.arange(1, n_guessers + 1),
    y=individual_guesses,
    mode='markers',
    name='Individual Guesses',
    marker=dict(color='lightblue', size=6, opacity=0.6)
))

# Running average
fig.add_trace(go.Scatter(
    x=np.arange(1, n_guessers + 1),
    y=ensemble_averages,
    mode='lines',
    name='Ensemble Average',
    line=dict(color='darkblue', width=3)
))

# True value
fig.add_hline(y=true_value, line_dash="dash", line_color="red", 
              annotation_text="True Value (500)")

fig.update_layout(
    title='The Wisdom of Crowds: Ensemble Averaging Reduces Error',
    xaxis_title='Number of Guessers (Models)',
    yaxis_title='Prediction',
    height=450,
    showlegend=True
)

fig.show()

**Interpretation**: As we add more guessers (models), the ensemble average converges toward the true value, even though individual guesses vary widely.

### 1.2 Types of Ensemble Methods

There are two main approaches to building ensembles:

| Method | Description | Examples |
|:-------|:------------|:---------|
| **Bagging** | Train models in parallel on bootstrapped samples | Random Forest, Bagged Trees |
| **Boosting** | Train models sequentially, each fixing previous errors | AdaBoost, Gradient Boosting, XGBoost |

In this module, we focus on **Bagging** and **Random Forests**. We'll cover boosting methods in a later module.

**Bagging** (Bootstrap Aggregating):
- Train many models independently
- Each model sees a different random sample of the data
- Combine predictions by voting (classification) or averaging (regression)

**Boosting**:
- Train models sequentially
- Each new model focuses on samples the previous models got wrong
- Combine with weighted voting

## 2. Bootstrap Aggregating (Bagging)

### 2.1 What is Bootstrapping?

**Bootstrapping** is a statistical technique where we create new datasets by sampling **with replacement** from the original data.

- Original dataset: [A, B, C, D, E]
- Bootstrap sample 1: [B, B, D, A, E] (B appears twice, C is missing)
- Bootstrap sample 2: [A, C, C, C, D] (C appears three times)

Key properties:
- Each bootstrap sample has the same size as the original
- On average, about **63.2%** of original samples appear in each bootstrap (some appear multiple times)
- About **36.8%** of samples are left out—these are called **Out-of-Bag (OOB)** samples

In [ ]:
# Demonstrate bootstrapping
np.random.seed(42)
original_data = np.array(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'])
n_samples = len(original_data)

print("Original Dataset:", list(original_data))
print("\nBootstrap Samples (with replacement):")
print("="*50)

for i in range(5):
    # Sample with replacement
    bootstrap_indices = np.random.choice(n_samples, size=n_samples, replace=True)
    bootstrap_sample = original_data[bootstrap_indices]
    
    # Find OOB samples
    oob_indices = set(range(n_samples)) - set(bootstrap_indices)
    oob_samples = original_data[list(oob_indices)]
    
    print(f"\nSample {i+1}: {list(bootstrap_sample)}")
    print(f"  Out-of-Bag: {list(oob_samples)} ({len(oob_samples)/n_samples*100:.0f}% left out)")

### 2.2 How Bagging Works

The bagging algorithm:

1. Create B bootstrap samples from the training data
2. Train one model on each bootstrap sample (in parallel)
3. Combine predictions:
   - **Classification**: Majority voting
   - **Regression**: Average predictions

```
Training Data
     |
     |---> Bootstrap Sample 1 ---> Tree 1 ---\
     |---> Bootstrap Sample 2 ---> Tree 2 ----\---> Aggregate ---> Final Prediction
     |---> Bootstrap Sample 3 ---> Tree 3 ----/
     |---> ...                 ---> ...   ---/
     |---> Bootstrap Sample B ---> Tree B --/
```

In [ ]:
# Visual representation of the bagging process
import plotly.graph_objects as go

fig = go.Figure()

# Training data box
fig.add_shape(type="rect", x0=0, y0=4, x1=2, y1=5,
              fillcolor="lightblue", line=dict(color="darkblue", width=2))
fig.add_annotation(x=1, y=4.5, text="Training Data<br>(n samples)", showarrow=False, font=dict(size=12))

# Bootstrap samples
for i, y_pos in enumerate([3.5, 2.5, 1.5, 0.5]):
    # Bootstrap sample box
    fig.add_shape(type="rect", x0=3, y0=y_pos-0.3, x1=5, y1=y_pos+0.3,
                  fillcolor="lightyellow", line=dict(color="orange", width=2))
    label = f"Bootstrap {i+1}" if i < 3 else "..."
    fig.add_annotation(x=4, y=y_pos, text=label, showarrow=False)
    
    # Tree box
    fig.add_shape(type="rect", x0=6, y0=y_pos-0.3, x1=8, y1=y_pos+0.3,
                  fillcolor="lightgreen", line=dict(color="green", width=2))
    tree_label = f"Tree {i+1}" if i < 3 else "..."
    fig.add_annotation(x=7, y=y_pos, text=tree_label, showarrow=False)
    
    # Arrows
    fig.add_annotation(x=3, y=y_pos, ax=2, ay=4.5, axref="x", ayref="y",
                      xref="x", yref="y", showarrow=True, arrowhead=2, arrowcolor="gray")
    fig.add_annotation(x=6, y=y_pos, ax=5, ay=y_pos, axref="x", ayref="y",
                      xref="x", yref="y", showarrow=True, arrowhead=2, arrowcolor="gray")

# Aggregation box
fig.add_shape(type="rect", x0=9, y0=1.7, x1=11, y1=2.3,
              fillcolor="lightcoral", line=dict(color="red", width=2))
fig.add_annotation(x=10, y=2, text="Aggregate<br>(Vote/Average)", showarrow=False)

# Final prediction
fig.add_shape(type="rect", x0=12, y0=1.7, x1=14, y1=2.3,
              fillcolor="lavender", line=dict(color="purple", width=2))
fig.add_annotation(x=13, y=2, text="Final<br>Prediction", showarrow=False)

# Arrows to aggregation and final
for y_pos in [3.5, 2.5, 1.5, 0.5]:
    fig.add_annotation(x=9, y=2, ax=8, ay=y_pos, axref="x", ayref="y",
                      xref="x", yref="y", showarrow=True, arrowhead=2, arrowcolor="gray")
fig.add_annotation(x=12, y=2, ax=11, ay=2, axref="x", ayref="y",
                  xref="x", yref="y", showarrow=True, arrowhead=2, arrowcolor="gray")

fig.update_layout(
    title='The Bagging Algorithm: Bootstrap Aggregating',
    xaxis=dict(visible=False, range=[-0.5, 14.5]),
    yaxis=dict(visible=False, range=[-0.5, 5.5]),
    height=400,
    showlegend=False
)

fig.show()

### 2.3 Variance Reduction Through Bagging

Bagging primarily reduces **variance** in predictions. If we have B independent models, each with variance $\sigma^2$, the variance of their average is:

$$\text{Var}(\bar{f}) = \frac{\sigma^2}{B}$$

In practice, the bootstrap samples aren't truly independent (they all come from the same data), but significant variance reduction still occurs.

**Why does this matter for student departure prediction?**
- A single decision tree might be very sensitive to which students are in the training set
- Bagging stabilizes predictions by averaging across many trees

In [ ]:
# Demonstrate variance reduction with increasing ensemble size
np.random.seed(42)

# Simulate predictions from different numbers of trees
n_experiments = 100
tree_counts = [1, 5, 10, 25, 50, 100, 200]
true_prob = 0.3  # True probability for a student

variance_by_count = []

for n_trees in tree_counts:
    ensemble_predictions = []
    for _ in range(n_experiments):
        # Simulate each tree's prediction (noisy around true probability)
        tree_preds = np.random.normal(true_prob, 0.15, n_trees)
        tree_preds = np.clip(tree_preds, 0, 1)
        # Ensemble prediction is the average
        ensemble_predictions.append(np.mean(tree_preds))
    variance_by_count.append(np.var(ensemble_predictions))

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=tree_counts,
    y=variance_by_count,
    mode='lines+markers',
    line=dict(color='darkblue', width=3),
    marker=dict(size=10)
))

fig.update_layout(
    title='Variance Reduction as Ensemble Size Increases',
    xaxis_title='Number of Trees in Ensemble',
    yaxis_title='Variance of Predictions',
    xaxis_type='log',
    height=400
)

fig.show()

**Key Insight**: Variance decreases rapidly at first, then levels off. This is why 100-500 trees is often sufficient for Random Forests.

## 3. Decision Trees: The Building Blocks

### 3.1 How Decision Trees Work

A **decision tree** makes predictions by asking a series of yes/no questions about the features.

For student departure prediction, a tree might ask:
- Is GPA_1 < 2.5?
  - Yes: Is DFW_RATE_1 > 0.3?
    - Yes: Predict DEPARTED
    - No: Is FIRST_GEN = Yes?
      - Yes: Predict DEPARTED
      - No: Predict RETAINED
  - No: Predict RETAINED

**How trees learn**: They recursively split the data to maximize "purity"—separating students who departed from those who were retained.

In [ ]:
# Visualize a simple decision tree structure
fig = go.Figure()

# Tree structure visualization
nodes = [
    # (x, y, text, color)
    (5, 5, "GPA_1 < 2.5?", "lightblue"),
    (2.5, 3.5, "DFW_RATE > 0.3?", "lightblue"),
    (7.5, 3.5, "RETAINED", "lightgreen"),
    (1, 2, "DEPARTED", "lightcoral"),
    (4, 2, "FIRST_GEN?", "lightblue"),
    (3, 0.5, "DEPARTED", "lightcoral"),
    (5, 0.5, "RETAINED", "lightgreen")
]

# Add nodes
for x, y, text, color in nodes:
    fig.add_shape(type="rect", x0=x-0.8, y0=y-0.4, x1=x+0.8, y1=y+0.4,
                  fillcolor=color, line=dict(color="darkgray", width=1))
    fig.add_annotation(x=x, y=y, text=text, showarrow=False, font=dict(size=10))

# Add edges
edges = [
    ((5, 4.6), (2.5, 3.9), "Yes"),
    ((5, 4.6), (7.5, 3.9), "No"),
    ((2.5, 3.1), (1, 2.4), "Yes"),
    ((2.5, 3.1), (4, 2.4), "No"),
    ((4, 1.6), (3, 0.9), "Yes"),
    ((4, 1.6), (5, 0.9), "No")
]

for (x1, y1), (x2, y2), label in edges:
    fig.add_trace(go.Scatter(
        x=[x1, x2], y=[y1, y2],
        mode='lines',
        line=dict(color='gray', width=2),
        showlegend=False
    ))
    mid_x, mid_y = (x1+x2)/2, (y1+y2)/2
    fig.add_annotation(x=mid_x, y=mid_y, text=label, showarrow=False,
                      font=dict(size=9, color='darkblue'))

fig.update_layout(
    title='Decision Tree for Student Departure Prediction',
    xaxis=dict(visible=False, range=[0, 10]),
    yaxis=dict(visible=False, range=[0, 6]),
    height=500,
    showlegend=False
)

fig.show()

### 3.2 Strengths and Weaknesses of Decision Trees

| Strengths | Weaknesses |
|:----------|:-----------|
| No assumptions about data distribution | High variance (small data changes = different tree) |
| Handle non-linear relationships | Prone to overfitting |
| Handle interactions naturally | Unstable (greedy splitting) |
| Easy to interpret | Can be biased toward high-cardinality features |
| No scaling required | |

**The key weakness**: A single decision tree has **high variance**. It can drastically change based on small changes to the training data.

**The solution**: Combine many trees via bagging to reduce variance while keeping the strengths!

## 4. Random Forests: Bagging + Randomness

### 4.1 The Random Forest Algorithm

A **Random Forest** is a bagged ensemble of decision trees with an additional twist: **feature randomness** at each split.

**Random Forest Algorithm**:

1. For b = 1 to B (number of trees):
   - Draw a bootstrap sample from the training data
   - Grow a decision tree on this sample, but:
     - At each split, only consider a **random subset of features**
     - Select the best split among this subset
   - Grow the tree fully (no pruning)

2. Aggregate predictions:
   - **Classification**: Majority vote across all trees
   - **Regression**: Average predictions across all trees

The name "Random Forest" comes from the randomness in:
1. **Rows**: Bootstrap sampling of observations
2. **Columns**: Random feature subset at each split

### 4.2 Feature Randomness: The Secret Ingredient

**Why add feature randomness?**

Without feature randomness, if one feature is very strong (e.g., GPA), all trees would split on it first. The trees would be highly correlated, limiting variance reduction.

By only considering a random subset of features at each split:
- Different trees use different features at the top
- Trees become less correlated
- Variance reduction is more effective

**Typical values for `max_features` (features considered at each split)**:
- Classification: $\sqrt{p}$ (square root of total features)
- Regression: $p/3$ (one-third of total features)

Where $p$ is the total number of features.

In [ ]:
# Demonstrate the effect of feature randomness
np.random.seed(42)

# Simulate tree correlation with and without feature randomness
n_sims = 1000
n_features = 10

# Without feature randomness: all trees use same top feature
# Results are highly correlated
predictions_no_random = np.random.normal(0.5, 0.1, (n_sims, 100))
for i in range(100):
    predictions_no_random[:, i] += np.random.normal(0, 0.05)  # Small perturbation

# With feature randomness: trees use different features
# Results are less correlated
predictions_with_random = np.random.normal(0.5, 0.1, (n_sims, 100))
for i in range(100):
    predictions_with_random[:, i] += np.random.normal(0, 0.15)  # Larger perturbation

# Calculate correlation matrices
corr_no_random = np.corrcoef(predictions_no_random[:, :20].T)
corr_with_random = np.corrcoef(predictions_with_random[:, :20].T)

fig = make_subplots(rows=1, cols=2, subplot_titles=(
    'Without Feature Randomness<br>(High Tree Correlation)',
    'With Feature Randomness<br>(Low Tree Correlation)'
))

fig.add_trace(go.Heatmap(z=corr_no_random, colorscale='RdBu', zmid=0,
                         showscale=False), row=1, col=1)
fig.add_trace(go.Heatmap(z=corr_with_random, colorscale='RdBu', zmid=0,
                         showscale=True, colorbar=dict(title='Correlation')), row=1, col=2)

fig.update_layout(
    title='Feature Randomness Reduces Tree Correlation',
    height=400
)
fig.update_xaxes(title='Tree Index')
fig.update_yaxes(title='Tree Index')

fig.show()

**Interpretation**: 
- Left: Without feature randomness, trees are highly correlated (red = high correlation)
- Right: With feature randomness, trees are more independent (blue = lower correlation)

Lower correlation means better variance reduction when averaging!

### 4.3 Making Predictions

For **classification** (our student departure problem):

- Each tree votes for a class (RETAINED or DEPARTED)
- The final prediction is the class with the most votes
- **Probability estimates**: Proportion of trees voting for each class

```
New Student: GPA=2.3, DFW_RATE=0.2, ...

Tree 1: DEPARTED (0.7 prob)
Tree 2: RETAINED (0.4 prob)
Tree 3: DEPARTED (0.8 prob)
Tree 4: DEPARTED (0.6 prob)
Tree 5: RETAINED (0.3 prob)
...
Tree 100: DEPARTED (0.55 prob)

Votes: 65 DEPARTED, 35 RETAINED
Final Prediction: DEPARTED
Probability: 0.65 (65% of trees voted DEPARTED)
```

In [ ]:
# Simulate voting across trees
np.random.seed(42)

n_trees = 100
# Simulate tree votes (0=Retained, 1=Departed)
# True probability is 0.65 for this student
tree_votes = np.random.binomial(1, 0.65, n_trees)

# Running vote proportion
vote_proportion = np.cumsum(tree_votes) / np.arange(1, n_trees + 1)

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=np.arange(1, n_trees + 1),
    y=vote_proportion,
    mode='lines',
    line=dict(color='darkblue', width=2),
    name='Vote Proportion'
))

fig.add_hline(y=0.5, line_dash="dash", line_color="gray",
              annotation_text="Decision Boundary (50%)")
fig.add_hline(y=0.65, line_dash="dash", line_color="red",
              annotation_text="True Probability (65%)")

fig.update_layout(
    title='Random Forest Voting: Probability Converges as Trees Increase',
    xaxis_title='Number of Trees',
    yaxis_title='Proportion Voting "Departed"',
    yaxis=dict(range=[0, 1]),
    height=400
)

fig.show()

print(f"Final vote: {sum(tree_votes)} Departed, {n_trees - sum(tree_votes)} Retained")
print(f"Predicted probability of departure: {sum(tree_votes)/n_trees:.2f}")

## 5. Random Forests vs. Logistic Regression

How do Random Forests compare to the logistic regression models we built in Modules 1-2?

| Aspect | Logistic Regression | Random Forest |
|:-------|:--------------------|:--------------|
| **Model Type** | Parametric (linear in log-odds) | Non-parametric (tree-based) |
| **Assumptions** | Linear decision boundary | No distributional assumptions |
| **Feature Interactions** | Must be explicitly added | Captured automatically |
| **Scaling Required** | Yes (affects regularization) | No |
| **Interpretability** | High (coefficients = log-odds) | Moderate (feature importance) |
| **Handles Non-linearity** | Limited | Excellent |
| **Risk of Overfitting** | Lower (especially with regularization) | Higher (but bagging helps) |
| **Training Speed** | Fast | Slower (many trees) |
| **Prediction Speed** | Very fast | Fast |
| **Works with Small Data** | Yes | Less reliable |

**When to use each**:

- **Logistic Regression**: When interpretability is crucial, relationships are approximately linear, or you have limited data
- **Random Forest**: When capturing complex patterns is more important than interpretability, or when you suspect non-linear relationships

In [ ]:
# Visualize linear vs non-linear decision boundaries
np.random.seed(42)

# Generate sample data with non-linear boundary
n_points = 200
X1 = np.random.uniform(0, 4, n_points)
X2 = np.random.uniform(0, 4, n_points)
# Non-linear boundary: circle
y = ((X1 - 2)**2 + (X2 - 2)**2 < 1.5).astype(int)
y = y + np.random.binomial(1, 0.1, n_points) - np.random.binomial(1, 0.1, n_points)
y = np.clip(y, 0, 1)

fig = make_subplots(rows=1, cols=2, subplot_titles=(
    'Logistic Regression<br>(Linear Boundary)',
    'Random Forest<br>(Non-linear Boundary)'
))

# Scatter plot
for col in [1, 2]:
    fig.add_trace(go.Scatter(
        x=X1[y==0], y=X2[y==0],
        mode='markers', marker=dict(color='blue', size=8),
        name='Retained', showlegend=(col==1)
    ), row=1, col=col)
    fig.add_trace(go.Scatter(
        x=X1[y==1], y=X2[y==1],
        mode='markers', marker=dict(color='red', size=8),
        name='Departed', showlegend=(col==1)
    ), row=1, col=col)

# Linear boundary (Logistic)
fig.add_trace(go.Scatter(
    x=[0, 4], y=[1, 3],
    mode='lines', line=dict(color='green', width=3, dash='dash'),
    name='Decision Boundary', showlegend=False
), row=1, col=1)

# Non-linear boundary (Random Forest) - circle
theta = np.linspace(0, 2*np.pi, 100)
fig.add_trace(go.Scatter(
    x=2 + 1.22*np.cos(theta), y=2 + 1.22*np.sin(theta),
    mode='lines', line=dict(color='green', width=3, dash='dash'),
    name='Decision Boundary', showlegend=False
), row=1, col=2)

fig.update_layout(
    title='Decision Boundary Comparison: Linear vs. Non-linear',
    height=400
)
fig.update_xaxes(title='GPA (scaled)', range=[0, 4])
fig.update_yaxes(title='DFW Rate (scaled)', range=[0, 4])

fig.show()

**Interpretation**: 
- Left: Logistic regression can only draw a straight line (or hyperplane in higher dimensions)
- Right: Random forests can capture complex, non-linear patterns

In student departure prediction, the relationship between features and departure probability may not be strictly linear. For example, the effect of GPA on departure risk might be different for students with very high vs. very low DFW rates.

## 6. Summary

In this notebook, we covered:

### Key Concepts

1. **Ensemble Learning**: Combining multiple models to achieve better performance
   - Works through the "wisdom of crowds" effect
   - Requires diverse models that make different errors

2. **Bagging (Bootstrap Aggregating)**:
   - Create bootstrap samples (sample with replacement)
   - Train one model per bootstrap sample
   - Aggregate via voting/averaging
   - Reduces variance while keeping bias similar

3. **Decision Trees**:
   - Make predictions via a series of if-then rules
   - Powerful but high variance (unstable)
   - Perfect building blocks for ensembles

4. **Random Forests**:
   - Bagged ensemble of decision trees
   - Added feature randomness at each split
   - Feature randomness decorrelates trees for better variance reduction

### Summary Table

| Concept | Key Point |
|:--------|:----------|
| Ensemble | Combine many weak learners into one strong learner |
| Bagging | Parallel training on bootstrap samples |
| Out-of-Bag | ~37% of data not in each bootstrap sample |
| Feature Randomness | Consider only subset of features at each split |
| max_features | Typically sqrt(p) for classification |

### Next Steps

In the next notebook, we will build a Random Forest pipeline for our student departure prediction problem using scikit-learn.

**Proceed to:** `3.2 Build a Random Forest Classification Model`